In [1]:
print("OK!")

OK!


In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


c:\Users\acer\anaconda3\envs\nari_env\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
PINECONE_API_KEY = "8e9658d0-c1cb-48ce-a2ae-2b136b8bf0b0"
#PINECONE_API_ENV = "gcp-starter"

In [4]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [5]:
extracted_data = load_pdf("data/")

In [6]:
# extracted_data

In [7]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [8]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 35


In [9]:
# text_chunks

In [10]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [11]:
embeddings = download_hugging_face_embeddings()

In [12]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [13]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [14]:
%pip install --upgrade --quiet  langchain-pinecone langchain-openai langchain

Note: you may need to restart the kernel to use updated packages.


In [15]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key='8e9658d0-c1cb-48ce-a2ae-2b136b8bf0b0')

In [17]:
index_name = "naritwa"

index = pc.Index(index_name)


In [18]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 7694}, 'real': {'vector_count': 322}},
 'total_vector_count': 8016}

In [19]:
%pip install --upgrade --quiet  langchain-pinecone langchain-openai langchain

Note: you may need to restart the kernel to use updated packages.


In [20]:
!pip install openai -q

In [21]:
# OPENAI_API_KEY='sk-proj-nqRs9vvwyOvdfVqSnYXUT3BlbkFJtQbVBwMv3FROgN2JdtEK'

In [22]:
# from langchain_community.document_loaders import TextLoader
# from langchain_openai import OpenAIEmbeddings
# from langchain_text_splitters import CharacterTextSplitter

# loader = TextLoader("data/file.txt")
# documents = loader.load()
# text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)
# docs = text_splitter.split_documents(documents)

# embeddingss = OpenAIEmbeddings(openai_api_key='ssk-proj-nqRs9vvwyOvdfVqSnYXUT3BlbkFJtQbVBwMv3FROgN2JdtEK')

In [23]:
import os
PINECONE_API_KEY='8e9658d0-c1cb-48ce-a2ae-2b136b8bf0b0'
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

In [24]:
from langchain_pinecone import PineconeVectorStore

index_name = "naritwa"


docsearch = PineconeVectorStore.from_documents(text_chunks, embeddings, index_name=index_name)

In [25]:
query = "Explain me about abortion laws of Nepal?"
docs = docsearch.similarity_search(query)
print(docs[0].page_content)

care
shall
have
to
provide
obstetric
care
in
a
respectful
manner.
The
non-governmental
and
private
health
institutions
fulfilling
the
standard
prescribed
by
the
Government
of
Nepal
shall
have
to
provide
emergency
obstetric
and
newborn
care.
In
case
the
health
institutions
are
not
able
to
manage
the
complications
that
arise
while
providing
service
by
them,
they
shall
have
to
refer
to
a
governmental
or
community
health
institution
to
the
extent
possible
and
to
a
non-governmental
and
private


In [ ]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [ ]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
# while True:
#     user_input=input(f"Input Prompt:")
#     result=qa({"query": user_input})
#     print("Response : ", result["result"])

In [26]:
from transformers import pipeline

In [27]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
c:\Users\acer\anaconda3\envs\nari_env\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [28]:
summarizer(docs[0].page_content, max_length=130, min_length=30, do_sample=False)

[{'summary_text': ' The Government of Nepal and the State of Nepal have responsibility for care for newborns . The Government has responsibility for the care of newborns in Nepal . The State has responsibility to provide care for all of its citizens .'}]